# Pool - how it manages elements?
<br>
<br>
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />
Sardana-Training by ALBA Synchrotron is licensed under the Creative Commons Attribution 4.0 International License.  
To view a copy of this license, visit http://creativecommons.org/licenses/by/4.0/.

Contents:
    
* Overview each Pool element type:
 * Element overview
 * Element API (some code snippets)
 * Tango element API

**Note1**: The code snippets use the testing utilities, beaware that their API may change

**Note2**: The code snippets use polling, it is better to use events

## Motor

* [Motor overview](http://www.sardana-controls.org/devel/overview/overview_motor.html#motor-overview)
* [Motor API reference](http://www.sardana-controls.org/devel/api/api_motor.html#sardana-motor-api)
* [Tango Motor API reference](http://www.sardana-controls.org/devel/api/sardana/tango/pool/Motor.html#sardana.tango.pool.Motor.Motor)

### Motor overview

* Interface was designed with physical motors in mind
* Interface is dynamic - only Position, State and Status attribute are mandatory
* Motor can be anything that can be changed e.g. power supply fits to a motor as well

### Motor API reference

* *position* and its relation with *dial position*, *offset* and *sign*
* *dial position* and its relation with *steps per unit*
* *state* and *limit switches*
* *velocity*, *acceleration*, *deceleration* and *base rate*
* *backlash* and *instability time*
* operations: *start move*, *stop* and *abort*

In [14]:
import time
from sardana.sardanadefs import State
from sardana.pool.test import BasePoolTestCase

util = BasePoolTestCase()
util.setUp()
ctrl = util.createController("motctrl01",
                      "DummyMotorController",
                      "DummyMotorController.py")
mot = util.createMotorElement(ctrl, "mot01", 1)

mot.start_move(100)
while mot.get_state() == State.Moving:
    time.sleep(0.01)
print mot.get_position()
util.tearDown()

Position(
       name = Position
    manager = mot01
       read = 100 at 2017-08-07 11:27:01.793232
      write = 100 at 2017-08-07 11:26:58.934780)



### Tango Motor API

* All the motor API are also available in Tango Motor API
* Extra interface
 * `DefinePosition` command
 * `MoveRelative` command \*
 * `SaveConfig` command \*
 * `Restore` command \*
 * `SimulationMode` attribute \* 
 * `State` and `Limit_switches` events
 * `Position` events
 
\* Not implemented or not fully tested

## Pseudo motor

* [Pseudo motor overview](http://www.sardana-controls.org/devel/overview/overview_pseudomotor.html)
* [Pseudo motor API reference](http://www.sardana-controls.org/devel/api/api_pseudomotor.html#sardana-pseudomotor-api)
* [Tango pseudo motor API reference](http://www.sardana-controls.org/devel/api/sardana/tango/pool/PseudoMotor.html#sardana.tango.pool.PseudoMotor.PseudoMotor)

### Pseudo motor overview
* Abstraction layer to provide a motor-like interface on top of motor(s) + calculation
* Examples are Slit's gap or offset pseudo motors
* Container of motor(s) so its state is the composition of physical motors' states

### Pseudo motor API reference
* *siblings*
* operations: *start move*, *stop* and *abort*
* *drift correction*:
 * why does it happen?
 * who is affected?
 * how is it solved?
 * TODO: move drift correction doc to the API

In [19]:
import time
from sardana.sardanadefs import State
from sardana.pool.test import BasePoolTestCase

util = BasePoolTestCase()
util.setUp()
mot_ctrl = util.createController("motctrl01",
                      "FastDummyMotorController",
                      "DummyMotorController.py")
left = util.createMotorElement(mot_ctrl, "left", 1)
right = util.createMotorElement(mot_ctrl, "right", 2)
slit_ctrl = util.createController("slitctrl01", "Slit", "Slit.py")
gap = util.createPMElement(slit_ctrl, "gap", 1, (left.id, right.id))
offset = util.createPMElement(slit_ctrl, "offset", 2, (left.id, right.id))

gap.start_move(0)
gap.drift_correction = False
while gap.get_state() == State.Moving:
    time.sleep(0.01)
offset.start_move(5)
while offset.get_state() == State.Moving:
    time.sleep(0.01)
print "gap:", gap.get_position().value, "offset:", offset.get_position().value
util.tearDown()

gap: 0.0 offset: 5.0


### Tango pseudo motor API

* All, apart of the siblings, the PseudoMotor API are also available in Tango Motor API 
* Extra interface
 * `CalcPhysical`, `CalcPseudo`, `CalcAllPhysical` and `CalcAllPseudo` commands
 * `MoveRelative` command \*
 * `SaveConfig` command \*
 * `Restore` command \*
 * `SimulationMode` attribute \* 
 * `State` events
 * `Position` events
 
\* Not implemented or not fully tested

### Other motion concepts

* Motion action - controls the motion execution in background (job - worker thread)
 * Consult states with "period" of 10 ms (`MotionLoop_SleepTime`)
 * Consults positions with every 10 (`MotionLoop_StatesPerPosition`)
 * Has emergency stop - when any of the motors is in Alarm state

### Other motion concepts

* Software limits - prevents motors to move beyond the limits
 * `set_lim` and `set_lm` macros
 * implemented using Tango attribute range
 * Moving more than one motor at a time does not verify software limits - see [#259](https://github.com/sardana-org/sardana/issues/259)
 * Software limits problems between motors and pseudomotors - see [#36](https://github.com/sardana-org/sardana/issues/36)
 * Solve inconsistencies between user position limits and dial position limits - see [#159](https://github.com/sardana-org/sardana/issues/159)
 * Changign steps per unit does not change software limits - see [#9](https://github.com/sardana-org/sardana/issues/9)

### Other motion concepts

* Motor group - the way to move multiple motors in a controled and sychronized way
 * Dynamically created element and Tango device (`_mg_ms_<random-number>`)
 
 * It is reused if another request arrives to move the same group of motors
 * Is not automatically cleaned up - manual cleaning may be convenient

### Other motion concepts

* Discrete pseudo motor - restricts the possible moves to a discrete set of states
 * Similar to IO register
 * Labels - names of the states
 * Calibration - relation between label and set point and eventual boundries of states
* TODO: demo - generic widget for interacting with moveables

## IO register

* [IO register overview](http://www.sardana-controls.org/devel/overview/overview_IOR.html)
* [IO register API reference](http://www.sardana-controls.org/devel/api/api_IOR.html)
* [Tango IO register API reference](http://www.sardana-controls.org/devel/api/sardana/tango/pool/IORegister.html#sardana.tango.pool.IORegister.IORegister)

### IO register overview

* Generic element to access hardware registers (`int`, `float` or `bool`)
* Write operation should be controlled - see [#21](https://github.com/sardana-org/sardana/issues/21)
* [TangoAttIORCtrl](https://sourceforge.net/u/zreszela/sardana-controllers/ci/master/tree/python/ioregister/TangoAttrIORCtrl.py) as alternative to discrete pseudo motor

## Counter/timer

* [Counter/timer overview](http://www.sardana-controls.org/devel/overview/overview_countertimer.html)
* [Counter/timer API reference](http://www.sardana-controls.org/devel/api/api_countertimer.html#sardana-countertimer-api)
* [Tango counter/timer API reference](http://www.sardana-controls.org/devel/api/sardana/tango/pool/CTExpChannel.html#sardana.tango.pool.CTExpChannel.CTExpChannel)

### Counter/timer overview

* Represents an experimental channel which acquisition result is a scalar value
* Interface was designed with hardware counter/timer devices in mind but fits well to electrometers or ADCs
* Acquisition is either synchronized on trigger and controlled by integration time or synchronized by gate

### Counter/timer API reference

* *value* attribute - stores the result of the last acquisition - see [SEP#17](https://github.com/reszelaz/sardana/blob/sep17/doc/source/sep/SEP17.md)
* Currently its acquisition is limited to the measurement group - see [#185](https://github.com/sardana-org/sardana/issues/185)
* operations: *start acquisition*, *stop*, *abort*

In [8]:
import time
from sardana.sardanadefs import State
from sardana.pool.test import BasePoolTestCase

util = BasePoolTestCase()
util.setUp()
ctrl = util.createController("ctctrl01",
                      "DummyCounterTimerController",
                      "DummyCounterTimerController.py")
ct = util.createCTElement(ctrl, "ct01", 1)
print ct.get_value()
ct.start_acquisition(0.1) # this does not work - see #185
while ct.get_state() == State.Moving:
    time.sleep(0.01)
print ct.get_value()
util.tearDown()

Value(
       name = Value
    manager = ct01
       read = 0.0 at 2017-08-02 15:58:42.088157
      write = -----)



KeyError: 'config'

## Tango counter/timer API

* All the counter/timer API are also available in Tango counter/timer API
* Extra interface
 * *Data* (in the future will be called *ValueBuffer*) attribute - events in continuous acquisition e.g. continuou scan (pipes should be evaluated instead of using attributes)
 * *Restore* \* 
 * *SimulationMode* attribute \*
 * *State* events
 * *Value* events

\* Not implemented or not fully tested

## 1D and 2D

* [1D overview](http://www.sardana-controls.org/devel/overview/overview_1D.html)
* [1D API reference](http://www.sardana-controls.org/devel/api/api_1D.html)
* [Tango 1D API reference](http://www.sardana-controls.org/devel/api/sardana/tango/pool/OneDExpChannel.html#sardana.tango.pool.OneDExpChannel.OneDExpChannel)
* [2D overview](http://www.sardana-controls.org/devel/overview/overview_2D.html)
* [2D API reference](http://www.sardana-controls.org/devel/api/api_2D.html)
* [Tango 2D API reference](http://www.sardana-controls.org/devel/api/sardana/tango/pool/OneDExpChannel.html#sardana.tango.pool.TwoDExpChannel.TwoDExpChannel)


### 1D and 2D overview

* Represents an experimental channel which acquisition result is:
 * 1D - a spectrum value
 * 2D - an image value
* Interface was designed with:
 * 1D - MCA, position sensitive detectors
 * 2D - CCD, photon counting area dectors
 in mind
* Acquisition is either synchronized on trigger and controlled by integration time or synchronized by gate - exactly the same as in case of counter/timer

### 1D and 2D API reference

* *value* attribute - stores the result of the last acquisition - see [SEP#17](https://github.com/reszelaz/sardana/blob/sep17/doc/source/sep/SEP17.md)
* TODO: *data source* attribute - unique identifier for the 1D data (value attribute)
* Currently its acquisition is limited to the measurement group - see [#185](https://github.com/sardana-org/sardana/issues/185)
* operations: *start acquisition*, *stop*, *abort*

### Tango 1D and 2D API

* All the 1D and 2D API are also available in Tango 1D and 2D API
* Extra interface
 * *Data* (in the future will be called *ValueBuffer*) attribute - events in continuous acquisition e.g. continuou scan
 * *Restore* \* 
 * *SimulationMode* attribute \*
 * *State* events
 * *Value*

\* Not implemented or not fully tested

## 0D

* [0D overview](http://www.sardana-controls.org/devel/overview/overview_0D.html)
* [0D API reference](http://www.sardana-controls.org/devel/api/api_0D.html)
* [Tango 0D API reference](http://www.sardana-controls.org/devel/api/sardana/tango/pool/ZeroDExpChannel.html#sardana.tango.pool.ZeroDExpChannel.ZeroDExpChannel)

### OD overview

* Is used to access any kind of device which returns a scalar value and which are not counter/timer
* Executes software sampling with "period" of 10 ms and performs calculation on the acquired buffer
* Acquisition is slave to counter/timer acquisition

### 0D API reference

* *value* and *accumulated value * attributes - calculation result
* *current value* - instant readout from the hardware
* *accumulation* - type of calculation on the buffer
* buffers: *accumulation* and *time* - raw samples and their timestamps
* currently its acquisition is limited to the measurement group - see [#185](https://github.com/sardana-org/sardana/issues/185)
* operations: *start acquisition*, *stop*, *abort*

### Tango 0D API

* All the 0D API are also available in Tango 0D API
* Extra interface
 * *Data* (in the future will be called *ValueBuffer*) attribute - events in continuous acquisition e.g. continuou scan
 * *ValueBuffer* is deprecated in favor of *AccumulationBuffer*
 * *Restore* \* 
 * *SimulationMode* attribute \*
 * *State* events
 * *Value* events

\* Not implemented or not fully tested

## Pseudo counter

* [Pseudo counter overview](http://www.sardana-controls.org/devel/overview/overview_pseudocounter.html)
* [Pseudo counter API reference](http://www.sardana-controls.org/devel/api/api_pseudocounter.html#sardana-pseudocounter-api)
* [Tango pseudo counter API reference](http://www.sardana-controls.org/devel/api/sardana/tango/pool/PseudoCounter.html#sardana.tango.pool.PseudoCounter.PseudoCounter)

### Pseudo counter overview

* Acts like an abstraction layer for any experimental channel or a set of channels allowing to perform calculations on them
* Examples is IoverI0 pseudo counter
* Container of channel(s) so its state is the composition of physical channels' states

### Pseudo counter API reference

* siblings
* operations: start move, stop and abort

### Tango pseudo counter API

* All, apart of the siblings, the pseudo counter API are also available in Tango pseudo counter API 
* Extra interface
 * `Data` (in the future will be called `ValueBuffer`) attribute - events in continuous acquisition e.g. continuou scan
 * `CalcPseudo` and `CalcAllPseudo` commands
 * `Restore` command \*
 * `SimulationMode` attribute \* 
 * `State` events
 
\* Not implemented or not fully tested

## Trigger/gate

* [Trigger/gate overview](http://www.sardana-controls.org/devel/overview/overview_triggergate.html)
* [Trigger/gate API reference](http://www.sardana-controls.org/devel/api/api_triggergate.html#sardana-triggergate-api)

### Trigger/gate
* Represents synchronization devices like for example the digital trigger and/or gate generators
* Their main role is to synchronize acquisition of the experimental channels
* May work in time and position domains

## Measurement group

* [Measurement group overview](http://www.sardana-controls.org/devel/overview/overview_measurementgroup.html)
* [Measurement group API reference](http://www.sardana-controls.org/devel/api/api_measurementgroup.html#sardana-measurementgroup-api)

### Measurement group overview

* Allows the user to access several data acquisition channels at the same time
* When creating a measurement group, the user compose it from:
 * Counter/Timer
 * 0D
 * 1D
 * 2D \*
 * Pseudo Counter
 * external attribute e.g. Tango \*
* Indirectly involves trigger/gate elements
* Container of element(s) so its state is the composition of its element states

\* Not compatible with continuous acquisition

### Measurement group API reference

* *configuration* - configuration of the controllers and channels to be used during the acquisition
* TODO: demo - expconf & online plot demo
* *synchronization* - it is composed from the group(s) of equidistant acquisitions described by the following parameters (in either time and/or position domain) - the same description is used for [trigger/gate synchronization](http://www.sardana-controls.org/devel/howto_controllers/howto_triggergatecontroller.html#synchronization-description)
 * initial point
 * initial delay
 * total interval
 * active interval
 * number of repetitions

### Measurement group API reference

* *integration time* and *monitor count* 
* *timer* - master timer
* *acquisition mode* - either `Timer` or `Monitor`.
* *latency time* 
* *moveable* - reference moveable

### Other acquisition concepts
* Acquisition action controls the acquisition execution in background (job - worker thread) - it is composed from sub-actions:
 * Software acquisition action and hardware acquisition action
   * Read states with "period" of 10 ms (AcqLoop_SleepTime)
   * Read values every 10 (AcqLoop_StatesPerPosition)
 * 0D acquisition
   * It is slave to software acquisition
   * Read values with periond of 10 ms (AcqLoop_SleepTime)
 * Synchronization
   * Read states with "period" of 200 ms (hardcoded)

### Other acquisition concepts
* Acquisition action starts the involved elements in a specific order:
 1. Experimental channels (first normal elements, then timers/monitors, at the end master timer/monitor)
 2. Synchronization elements (software synchronizer and/or trigger/gate elements)